# Data Loader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torchvision import transforms
import torchvision
import numpy as np

from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import glob

import matplotlib.pyplot as plt

class MyDataset_gopro(Dataset):
  def __init__(self, path, train=True, transform=None):
    self.path=path
    self.blurPathList = []
    self.sharpPathList = []
    if train:
      for i in range(1,8):
        self.blurPathList.append(path+"train/"+str(i)+"/blur")
        self.sharpPathList.append(path+"train/"+str(i)+"/sharp") 
    else:
      for i in range(1,2):
        self.blurPathList.append(path+"test/"+str(i)+"/blur")
        self.sharpPathList.append(path+"test/"+str(i)+"/sharp")


    self.transform = transform
    self.blurImgList=[]
    self.sharpImgList=[]
    if train:
      for i in range(7): 
        self.blurImgList += glob.glob(self.blurPathList[i]+'/*.png')
      for i in range(7):  
        self.sharpImgList += glob.glob(self.sharpPathList[i]+'/*.png')
    else:
      for i in range(1): 
        self.blurImgList += glob.glob(self.blurPathList[i]+'/*.png')
      for i in range(1):  
        self.sharpImgList += glob.glob(self.sharpPathList[i]+'/*.png')

  
  def __len__(self):
    return len(self.blurImgList)

  def __getitem__(self,idx):
    blurImgPath = self.blurImgList[idx]
    sharpImgPath = self.sharpImgList[idx]
    blurImg = Image.open(blurImgPath)
    sharpImg = Image.open(sharpImgPath)

    blurImg=blurImg.crop((300,100,812,612))
    sharpImg=sharpImg.crop((300,100,812,612))


    if self.transform is not None:
      blurImg = self.transform(blurImg)
      sharpImg = self.transform(sharpImg)
    
    return blurImg,sharpImg

In [ ]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        # transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)

goPropath = '/content/drive/MyDrive/dataset_gopro/'
train_dataset = MyDataset_gopro(path=goPropath,train=True, transform=transform)

In [ ]:
train_dataloader = DataLoader(dataset=train_dataset,
                        batch_size=50,
                        shuffle=True,
                        drop_last=True)

train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

#########
len(train_dataloader) ## 총 배치의 수

Feature batch shape: torch.Size([50, 3, 512, 512])
Labels batch shape: torch.Size([50, 3, 512, 512])


12

# Models

## Model1

In [4]:
import torch.nn.init

USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 Flase
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용, 아니면 CPU tkdyd
print("다음 기기로 학습합니다: ", device)

if device == 'cuda':
  torch.cuda.manual_seed_all(777)


# Layer1 : (batch_size,3,512,512) => (batch_size,32,256,256)
# Conv2d(in_channel=3, outchannel=32, kernelsize=3, stride=1, padding=1)
# ReLU
# Pool (kernel_size=2, stride=2)

# Layer2 : (batch_size,32,256,256) => (batch_size,32,256,256)
# Conv2d(in_channel=3, outchannel=32, kernelsize=3, stride=1, padding=1)
# ReLU

# Layer3 : (batch_size,32,256,256) => (batch_size,32,256,256)
# Conv2d(in_channel=3, outchannel=32, kernelsize=3, stride=1, padding=1)
# ReLU

# Layer4 : (batch_size,32,256,256) => (batch_size,3,512,512)
# ConTranspose2d(in_channel=3, outchannel=32, kernelsize=22, stride=2, padding=10)


class MyCNN(torch.nn.Module):
  def __init__(self):
    super(MyCNN, self).__init__()
    self.keep_prob = 0.5
    #첫번째층 (3,512,512) => (32,256,256)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )
    # 두번째층 (32,256,256) => (32,256,256)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU()
    )
    # 세번째층 (32,256,256) => (32,256,256)
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU()
    )
    # 네번째층 (32,256,256) => (3,512,512)
    self.layer4 = torch.nn.Sequential(
        torch.nn.ConvTranspose2d(32,3,22,2,10)
    )

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    return out

다음 기기로 학습합니다:  cuda


## Model2

In [ ]:
import torch.nn.init

USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 Flase
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용, 아니면 CPU tkdyd
print("다음 기기로 학습합니다: ", device)

if device == 'cuda':
  torch.cuda.manual_seed_all(777)


# Layer1 : (batch_size,3,512,512) => (batch_size,32,256,256)
# Conv2d(in_channel=3, outchannel=32, kernelsize=32, stride=2, padding=10)
# ReLU

# Layer2 : (batch_size,32,256,256) => (batch_size,32,256,256)
# Conv2d(in_channel=3, outchannel=32, kernelsize=3, stride=1, padding=1)
# ReLU

# Layer3 : (batch_size,32,256,256) => (batch_size,32,256,256)
# Conv2d(in_channel=3, outchannel=32, kernelsize=3, stride=1, padding=1)
# ReLU

# Layer4 : (batch_size,32,256,256) => (batch_size,3,512,512)
# ConTranspose2d(in_channel=3, outchannel=32, kernelsize=22, stride=2, padding=10)

class MyCNN2(torch.nn.Module):
  def __init__(self):
    super(MyCNN2, self).__init__()
    self.keep_prob = 0.5
    #첫번째층 (3,512,512) => (32,256,256)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(3, 32, kernel_size=22, stride=2, padding=10),
        torch.nn.ReLU(),
    )
    # 두번째층 (32,256,256) => (32,256,256)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU()
    )
    # 세번째층 (32,256,256) => (32,256,256)
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU()
    )
    # 네번째층 (32,256,256) => (3,512,512)
    self.layer4 = torch.nn.Sequential(
        torch.nn.ConvTranspose2d(32,3,22,2,10)
    )

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    return out

## Model3

In [ ]:
import torch.nn.init

USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 Flase
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용, 아니면 CPU tkdyd
print("다음 기기로 학습합니다: ", device)

if device == 'cuda':
  torch.cuda.manual_seed_all(777)


# Layer1 : (batch_size,3,512,512) => (batch_size,32,256,256)
# Conv2d(in_channel=3, outchannel=32, kernelsize=32, stride=2, padding=10)
# ReLU

# Layer2 : (batch_size,32,256,256) => (batch_size,32,256,256)
# Conv2d(in_channel=32, outchannel=32, kernelsize=3, stride=1, padding=1)
# ReLU

# Layer3 : (batch_size,32,256,256) => (batch_size,4,256,256)
# Conv2d(in_channel=32, outchannel=16, kernelsize=1, stride=1, padding=0)
# ReLU

# Layer4 : (batch_size,4,256,256) => (batch_size,32,256,256)
# Conv2d(in_channel=32, outchannel=32, kernelsize=3, stride=1, padding=1)
# ReLU

# Layer5 : (batch_size,32,256,256) => (batch_size,3,512,512)
# ConTranspose2d(in_channel=3, outchannel=32, kernelsize=22, stride=2, padding=10)

class MyCNN3(torch.nn.Module):
  def __init__(self):
    super(MyCNN3, self).__init__()
    self.keep_prob = 0.5
    #첫번째층 (3,512,512) => (16,256,256)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )
    # 두번째층 (16,256,256) => (32,256,256)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU()
    )
    # 세번째층 (32,256,256) => (4,256,256)
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 4, kernel_size=1, stride=1, padding=0),
        torch.nn.ReLU()
    )
    # 네번째층 (4,256,256) => (32,256,256)
    self.layer4 = torch.nn.Sequential(
        torch.nn.Conv2d(4, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU()
    )
    # 다섯번째층 (32,256,256) => (3,512,512)
    self.layer5 = torch.nn.Sequential(
        torch.nn.ConvTranspose2d(32,3,22,2,10)
    )

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.layer5(out)
    return out

다음 기기로 학습합니다:  cuda


## Model4

In [ ]:
# bn1 = torch.nn.BatchNorm1d(32)

# 학습

In [ ]:
# MyCNN 모델 정의
model = MyCNN3().to(device)

learning_rate = 0.001
training_epochs = 20

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_batch = len(train_dataloader)
print('총 배치의 수 : {}'.format(total_batch))


for epoch in range(training_epochs):
  batch_idx=0
  avg_cost = 0

  for X, Y in train_dataloader: # 미니 배치 단위로 꺼내옴
  
    X = X.to(device)
    Y = Y.to(device)
    
    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
    print('[Epoch: {:>4}] batch = {:>4}'.format(epoch + 1, batch_idx + 1))
    batch_idx += 1
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
  if epoch%2==0:
    torch.save(model.state_dict(), '/content/drive/MyDrive/Model3/model_state_dict'+str(epoch+1)+'.pt')

# 모델 저장하기
# /content/drive/MyDrive/Model
torch.save(model.state_dict(), '/content/drive/MyDrive/Model3/'+'model_state_dict.pt')

# Validation set으로 테스트
  - mse 구하기
  - batch_size = 25

In [6]:
valid_dataset = MyDataset_gopro(path=goPropath,train=False, transform=transform)

NameError: ignored

In [ ]:
valid_dataloader = DataLoader(dataset=valid_dataset,
                        batch_size=25,
                        shuffle=False,
                        drop_last=True)
valid_features, valid_labels = next(iter(valid_dataloader))
print(f"Feature batch shape: {valid_features.size()}")
print(f"Labels batch shape: {valid_labels.size()}")

Feature batch shape: torch.Size([25, 3, 512, 512])
Labels batch shape: torch.Size([25, 3, 512, 512])


In [ ]:
model_Val = MyCNN3().to(device)
model_Val.load_state_dict(torch.load('/content/drive/MyDrive/Model3/model_state_dict.pt'))
model_Val.eval()


criterion = torch.nn.MSELoss()
total_batch = len(valid_dataloader)

avg_cost=0

for X, Y in valid_dataloader: # 미니 배치 단위로 꺼내옴
  X = X.to(device)
  Y = Y.to(device)

  hypothesis = model_Val(X)
  cost = criterion(hypothesis, Y)

  avg_cost += cost / total_batch
print(f"avg cost: {avg_cost}")

avg cost: 0.007290552835911512


# 과제 사진 처리 및 저장
1. 이미지 oprn
2. open한 이미지

In [1]:
import torch
from torchvision import transforms
import torchvision
import numpy as np

from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import glob

import matplotlib.pyplot as plt

class MyDataset_gopro_Test(Dataset):
  def __init__(self, transform=None):
    self.transform = transform
    self.blurImgList=[]
    self.blurImgList += glob.glob('/content/drive/MyDrive/testImg/*.png')

  
  def __len__(self):
    return len(self.blurImgList)

  def __getitem__(self,idx):
    blurImgPath = self.blurImgList[idx]
    blurImg = Image.open(blurImgPath)


    if self.transform is not None:
      blurImg = self.transform(blurImg)
    
    return blurImg

In [2]:
transform_test = transforms.Compose(
    [
        transforms.ToTensor(),
        # transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)
test_dataset = MyDataset_gopro_Test(transform=transform_test)

In [5]:
# /content/drive/MyDrive/testImg/*.png : 과제 사진
# /content/drive/MyDrive/result/*.png : 내 결과
topilimage = transforms.ToPILImage()
totensor = transforms.ToTensor()


# Valid dataloader todtjd
test_dataloader = DataLoader(dataset=test_dataset,
                        batch_size=30,
                        shuffle=False,
                        drop_last=True)
test_features = next(iter(test_dataloader))

# 이용할 모델
model_test = MyCNN().to(device)
model_test.load_state_dict(torch.load('/content/drive/MyDrive/Model1/model1.pt'))
model_test.eval()

result = model_test(test_features.to(device))

for i in range(30):
  topilimage(result[i]).save('/content/drive/MyDrive/resultImg/'+str(i+1)+'.png')
